In [230]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [231]:
Bundes_liga_19_20 = pd.read_csv("data_auto/Bundes_lige2019-2020.csv")
Bundes_liga_20_21 = pd.read_csv("data_auto/Bundes_lige2020-2021.csv")
Bundes_liga_21_22 = pd.read_csv("data_auto/Bundes_lige2021-2022.csv")
Bundes_liga_22_23 = pd.read_csv("data_auto/Bundes_lige2022-2023.csv")


In [232]:
display(Bundes_liga_19_20.shape)
display(Bundes_liga_20_21.shape)
display(Bundes_liga_21_22.shape)
display(Bundes_liga_22_23.shape)
n = [Bundes_liga_19_20, Bundes_liga_20_21, Bundes_liga_21_22, Bundes_liga_22_23]

(308, 74)

(308, 73)

(308, 73)

(308, 73)

In [233]:
def preobr(df):
    if 'Unnamed: 0.1' in list(df.columns):
        df = df.drop('Unnamed: 0.1',axis=1)
    if 'Unnamed: 0' in list(df.columns):
        df = df.drop('Unnamed: 0',axis=1)
    
    df = df.drop('Матчи 1',axis=1)
    df = df.drop('Матчи 2',axis=1)
    df = df.astype({'счет1': 'int32'})
    df = df.astype({'счет2': 'int32'})

    df["тотал"] = df["счет1"] + df["счет2"]
    df["Дата"] = pd.to_datetime(df['Дата'],dayfirst=True)
    df['Температура'] = df['Температура'].apply(lambda x: int(x[1:-2]))
    
    mask_1 = df["id"] == 14898101
    df['Офсайды  (соперник) 2'][mask_1] = 4.00
    df['Офсайды 2'][mask_1] = 1.00
        
    df["Point_1"] = 0
    df["Point_2"] = 0
    df["liga"] = df["общая информация"].apply(lambda x: x.split(",")[0])
    df["Tour"] = df["общая информация"].apply(lambda x: x.split(",")[1])
    
    df = df[df["Tour"] != " стыковые матчи"]
    df = df[df["Tour"] != " понижение/повышение - финал"]
    df["Tour"] = df["Tour"].apply(lambda x: int(x.strip()[:-6]))
    
    df = df.astype({'Офсайды  (соперник) 2': 'float32'})
    df = df.astype({'Офсайды 2': 'float32'})

    df["Сухие матчи 1"] = df["Сухие матчи 1"].apply(lambda x: int(x.split()[-1]))
    df["Сухие матчи 2"] = df["Сухие матчи 2"].apply(lambda x: int(x.split()[0]))

    df["обе забьют 1"] = df["обе забьют 1"].apply(lambda x: int(x.split()[-1]))
    df["обе забьют 2"] = df["обе забьют 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал больше 2.5 1"] = df["Тотал больше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал больше 2.5 2"] = df["Тотал больше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал меньше 2.5 1"] = df["Тотал меньше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал меньше 2.5 2"] = df["Тотал меньше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Поражения 1"] = df["Поражения 1"].apply(lambda x: int(x.split()[-1]))
    df["Поражения 2"] = df["Поражения 2"].apply(lambda x: int(x.split()[0]))

    df["Ничьи 1"] = df["Ничьи 1"].apply(lambda x: int(x.split()[-1]))
    df["Ничьи 2"] = df["Ничьи 2"].apply(lambda x: int(x.split()[0]))

    df["Победы 1"] = df["Победы 1"].apply(lambda x: int(x.split()[-1]))
    df["Победы 2"] = df["Победы 2"].apply(lambda x: int(x.split()[0]))

    df["Отдых: дни (часы) 1"] = df["Отдых: дни (часы) 1"].apply(lambda x: float(x.split()[-1]))
    df["Отдых: дни (часы) 2"] = df["Отдых: дни (часы) 2"].apply(lambda x: float(x.split()[0]))
    return df

In [234]:
Bundes_liga_19_20_cl = preobr(Bundes_liga_19_20)
Bundes_liga_20_21_cl = preobr(Bundes_liga_20_21)
Bundes_liga_21_22_cl = preobr(Bundes_liga_21_22)
Bundes_liga_22_23_cl = preobr(Bundes_liga_22_23)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8064\3737429074.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Офсайды  (соперник) 2'][mask_1] = 4.00
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8064\3737429074.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Офсайды 2'][mask_1] = 1.00
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8064\3737429074.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Офсайды  (соперник)

In [235]:
n = list(Bundes_liga_20_21_cl["Команда1"].value_counts().index)
d = dict()
for i in n:
    d.update({i:0})
d

{'Бавария': 0,
 'Боруссия Дортмунд': 0,
 'Аугсбург': 0,
 'Шальке': 0,
 'Герта': 0,
 'Фрайбург': 0,
 'Байер': 0,
 'Боруссия М': 0,
 'Майнц': 0,
 'РБ Лейпциг': 0,
 'Штутгарт': 0,
 'Вердер': 0,
 'Кёльн': 0,
 'Унион Берлин': 0,
 'Айнтрахт Франкфурт': 0,
 'Вольфсбург': 0,
 'Хоффенхайм': 0,
 'Арминия Билефельд': 0}

In [236]:
n = list(Bundes_liga_20_21_cl["Команда1"].value_counts().index)
d = dict()
for i in n:
    d.update({i:0})
d
df = Bundes_liga_20_21_cl.sort_values(by="Tour",axis=0)
for i in list(df["id"]):
    num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
    
    df.loc[df["id"] == i ,["Point_1"]] = d[num[0]]
    df.loc[df["id"] == i ,["Point_2"]]  = d[num[2]]
    
    if num[1] > num[3]:
        d[num[0]] +=  3
    elif num[1] < num[3]:
        d[num[2]] +=  3
    else:
        d[num[0]] +=  1
        d[num[2]] +=  1

In [240]:
display(df[df["Tour"] == 34][["Команда1", "счет1", "Point_1"]].sort_values(by="Point_1"))
display(df[df["Tour"] == 34][["Команда2", "счет2", "Point_2"]].sort_values(by="Point_2"))

,Команда1,счет1,Point_1
8,Кёльн,1,30
9,Вердер,2,31
4,Хоффенхайм,2,40
10,Штутгарт,0,45
7,Унион Берлин,2,47
6,Айнтрахт Франкфурт,3,57
5,Вольфсбург,2,61
3,Боруссия Дортмунд,3,61
2,Бавария,5,75


,Команда2,счет2,Point_2
8,Шальке,0,16
10,Арминия Билефельд,2,32
4,Герта,1,35
5,Майнц,3,36
2,Аугсбург,2,36
6,Фрайбург,1,45
9,Боруссия М,4,46
3,Байер,1,52
7,РБ Лейпциг,1,65


In [246]:
list(d.values())

[78, 64, 36, 16, 35, 45, 52, 49, 39, 65, 45, 31, 33, 50, 60, 61, 43, 35]

In [253]:
countries_pd = pd.DataFrame({
    'счет': list(d.values())
})

countries_pd.index = list(d.keys())
countries_pd.sort_values(by="счет",axis=0,ascending=False)

,счет
Бавария,78
РБ Лейпциг,65
Боруссия Дортмунд,64
Вольфсбург,61
Айнтрахт Франкфурт,60
Байер,52
Унион Берлин,50
Боруссия М,49
Фрайбург,45
Штутгарт,45
